<a href="https://colab.research.google.com/github/ayushcodes13/Baymax/blob/main/AIDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install trl==0.14.0 peft==0.14.0 xformers==0.0.28.post3
!pip install torch==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu124

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-j7donm7r/unsloth_a85d1785c96a43bf8cb6f3cf973eed1a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-j7donm7r/unsloth_a85d1785c96a43bf8cb6f3cf973eed1a
  Resolved https://github.com/unslothai/unsloth.git to commit 73dd1227b09ab5e4d8a151e15410a643c99a1e82
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://download.pytorch.org/whl/cu124


In [8]:
!nvidia-smi


Sat Aug 30 14:14:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P0             30W /   70W |     142MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [10]:
from google.colab import drive
drive.mount('/content/drive')
# ensure TrainingArguments output_dir points to /content/drive/MyDrive/...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

In [12]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [13]:
import torch
print('CUDA availabe:', torch.cuda.is_available())
print("GPU Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA availabe: True
GPU Device: Tesla T4


In [14]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
dtype = None
load_in_4bit = True

model, tokenizer =FastLanguageModel.from_pretrained(
    model_name = model_name,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [15]:
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
{}

### Answer:
{}
"""


In [16]:
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])



**Step-by-Step Explanation:**

1. **Understanding the Symptoms:**
   - The patient is a 61-year-old woman with a history of involuntary urine loss during activities such as coughing or sneezing. This suggests a potential issue with urinary continentia, specifically stress urinary incontinence (SUI), as it typically occurs during activities that increase abdominal pressure.

2. **Gynecological Exam and Q-Tip Test:**
   - The gynecological exam likely included a pelvic examination to assess the urethra and bladder.
   - The Q-tip test evaluates the closure mechanism of the urethral sphincter. If the test shows positive (the Q-tip remains above the pH of urine, indicating sphincter function is intact), it supports the diagnosis of SUI over urge urinary incontinence (UUI).

3. **Cystometry (Bladder Diary):**
   - Cystometry is a diagnostic tool that measures bladder capacity, residual volume, and detrusor contractions.
   - In SUI, the primary issue is an overactive detrusor muscle durin

In [35]:
# Load the full dataset
medical_dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    trust_remote_code=True
)

# Reduce to 5000 examples (random sampling)
medical_dataset["train"] = medical_dataset["train"].shuffle(seed=42).select(range(5000))

In [18]:
medical_dataset["train"][1]


KeyError: "Invalid key: 1. Please first select a split. For example: `my_dataset_dictionary['train'][1]`. Available splits: ['train']"

In [36]:
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [37]:

### Finetuning
# Updated training prompt style to add  tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:

{}

{}"""

In [38]:
MAX_LENGTH = 2048
def preprocess_input_data(examples):
  inputs = examples["Question"]
  cots = examples.get("Complex_CoT", [""] * len(inputs))
  outputs = examples["Response"]
  texts = []
  for inp, cot, out in zip(inputs, cots, outputs):
    prompt = train_prompt_style.format(inp, cot, out) + EOS_TOKEN
    if len(tokenizer.encode(prompt)) > MAX_LENGTH:
       prompt = tokenizer.decode(tokenizer.encode(prompt)[-MAX_LENGTH:])  # keep last tokens if very long
    texts.append(prompt)
  return {"texts": texts}


In [39]:
finetune_dataset = medical_dataset.map(
    preprocess_input_data,
    batched=True,
    remove_columns=['Question', 'Complex_CoT', 'Response']  # explicitly remove
)

print(finetune_dataset["train"]["texts"][0])


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
In the instrument formula for a Gingival Margin Trimmer (GMT) used during cavity preparation, what is the second number representing the angle of the cutting edge when access to the distal gingival margin is achieved?

### Response:

Alright, so a Gingival Margin Trimmer, or GMT for short, is some sort of dental tool used during cavity prep. I need to figure out what that second number in its formula really means, especially when working with the distal gingival margin. Let's start with the basics about these

In [40]:
model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)


In [41]:
# Add this before creating the trainer
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate


In [42]:
finetune_dataset["train"] = finetune_dataset["train"].rename_column("texts", "text")


In [43]:
trainer = SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=finetune_dataset["train"],
    dataset_text_field="text",  # now matches
    max_seq_length=max_sequence_length,
    dataset_num_proc=1,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/baymax/outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=2,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=50,
        save_strategy="steps",
        save_steps=500,
        save_total_limit=3,
    )
)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [44]:
# Setup WANDB
from google.colab import userdata
wnb_token = userdata.get("WANDB_API_TOKEN")
# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [45]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 2 | Total steps = 626
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
wandb.finish()

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# Load base + adapter
model = AutoModelForCausalLM.from_pretrained(base_model, load_in_4bit=False, device_map="auto")
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/baymax/deepseek_lora_v1")

# Merge LoRA weights into base model
model = model.merge_and_unload()

# Save as a standalone full model
save_path = "/content/drive/MyDrive/baymax/deepseek_merged_v1"
model.save_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.save_pretrained(save_path)


In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/baymax/deepseek_merged_v1")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/baymax/deepseek_merged_v1")


In [ ]:
# Step10: Testing after fine-tuning
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])
